# กรมบังคับคดี

In [2]:
#กรมบังคับคดี
from urllib.parse import quote
from bs4 import BeautifulSoup
import requests
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import csv
import sys
import pandas as pd
from datetime import datetime
from csv import DictWriter
import json
import os.path

def deed_no_from_url(i):
    i = i.split('&')[-2].split('=')[-1].replace('%20',',').replace('-',',').replace('%',',').replace('.','').replace('(',',').replace(')',',').split(',')
    d = []
    for ii in i:
        if ii != '' and len(ii) >2:
            d.append(ii)
    d = str(d).replace('[','').replace(']','').replace(' ','').replace("'",'')
    return d
    
def read_table_data(province,page):
    prov_percen = find_percen_province(province)

    print('page : ',page)
    url = 'http://asset.led.go.th/newbidreg/asset_search_province.asp?search=ok&search_asset_type_id=&search_tumbol=&search_ampur=&search_province=' + prov_percen + '&search_sub_province=&search_price_begin=&search_price_end=&search_bid_date=&search_rai=&search_quaterrai=&search_wa=&search_rai_if=1&search_quaterrai_if=1&search_wa_if=1&search_status=1&search_person1=&page='+str(page)
    driver.get(url)
    
    max_row = len(driver.find_element_by_xpath('/html/body/div[4]/div/div[2]/div[2]/table').text.split('\n'))-12
    for row in range(1,max_row+1):
                
        lot = driver.find_element_by_xpath(f'/html/body/div[4]/div/div[2]/div[2]/table/tbody/tr[{row}]/td[1]').text
        sell_order = driver.find_element_by_xpath(f'/html/body/div[4]/div/div[2]/div[2]/table/tbody/tr[{row}]/td[2]').text
        case_id = driver.find_element_by_xpath(f'/html/body/div[4]/div/div[2]/div[2]/table/tbody/tr[{row}]/td[3]').text
        type_prop= driver.find_element_by_xpath(f'/html/body/div[4]/div/div[2]/div[2]/table/tbody/tr[{row}]/td[4]').text
        size1 = driver.find_element_by_xpath(f'/html/body/div[4]/div/div[2]/div[2]/table/tbody/tr[{row}]/td[5]').text
        size2 = driver.find_element_by_xpath(f'/html/body/div[4]/div/div[2]/div[2]/table/tbody/tr[{row}]/td[6]').text
        size3 = driver.find_element_by_xpath(f'/html/body/div[4]/div/div[2]/div[2]/table/tbody/tr[{row}]/td[7]').text
        price = driver.find_element_by_xpath(f'/html/body/div[4]/div/div[2]/div[2]/table/tbody/tr[{row}]/td[8]').text
        tumbon = driver.find_element_by_xpath(f'/html/body/div[4]/div/div[2]/div[2]/table/tbody/tr[{row}]/td[9]').text.strip()
        aumper = driver.find_element_by_xpath(f'/html/body/div[4]/div/div[2]/div[2]/table/tbody/tr[{row}]/td[10]').text.strip()
        page_row = str(page)+'/'+str(row)
        
        df = pd.read_csv(f'{path}/{file_name}', delimiter ='|')
        
        if not page_row in list(df['page']):
            driver.find_element_by_xpath(f'/html/body/div[4]/div/div[2]/div[2]/table/tbody/tr[{row}]').click()

            driver.switch_to.window(driver.window_handles[1])
            clicked_url = driver.current_url
            print('clicked_url',clicked_url)

            Law_Court_ID = clicked_url.split('&')[-3].split('=')[-1]
            deed_no = deed_no_from_url(clicked_url)
            addrno = clicked_url.split('&')[-1].split('=')[-1]        

            bit_time = []
            for i in range(6):
                try:
                    bit_time.append(driver.find_element_by_xpath(f'/html/body/div[1]/div/div/div[6]/div/table/tbody/tr[{i+1}]').text.strip())
                except:
                    bit_time.append(driver.find_element_by_xpath(f'/html/body/div[1]/div/div/div[4]/div/table/tbody/tr[{i+1}]').text.strip())

            dict_data = {
                'case_id': case_id,
                'page': page_row,
                'lot': lot,
                'sell_order': sell_order,
                'type_prop': type_prop,
                'size1' : size1,
                'size2' : size2,
                'size3' : size3,
                'price' : float(price.replace(',','')),
                'tumbon' : tumbon,
                'aumper': aumper,
                'Law_Court_ID': Law_Court_ID,
                'deed_no': deed_no,
                'addrno': addrno,
                'bit_time1': bit_time[0],
                'bit_time2': bit_time[1],
                'bit_time3': bit_time[2],
                'bit_time4': bit_time[3],
                'bit_time5': bit_time[4],
                'bit_time6': bit_time[5],
                'link': clicked_url,
                'timestamp' : str(datetime.now()),
                'province': province

            }

            driver.close()
            driver.switch_to.window(driver.window_handles[0])

            print(dict_data)
            print('\n\n\n')

            with open(f'{path}/{file_name}', 'a', encoding='UTF8') as f:

                data = []
                for i in header:
                    data.append(dict_data[i])

                writer = csv.writer(f,delimiter ='|')
                writer.writerow(data)
        else:
            print(case_id,page_row,'exist data--->pass')

def find_percen_province(province):
    url = 'http://asset.led.go.th/newbidreg/'
    driver.get(url)
 
    driver.find_element_by_xpath('/html/body/div[4]/div/div/div/div/div[3]/div/div[1]/table/tbody/tr/td[2]/select').send_keys(province)

    prov_percen = driver.current_url.split('=')[-1]

    return prov_percen

def find_maxpage(province):
    prov_percen = find_percen_province(province)
    
    page = 1
    url = 'http://asset.led.go.th/newbidreg/asset_search_province.asp?search=ok&search_asset_type_id=&search_tumbol=&search_ampur=&search_province=' + prov_percen + '&search_sub_province=&search_price_begin=&search_price_end=&search_bid_date=&search_rai=&search_quaterrai=&search_wa=&search_rai_if=1&search_quaterrai_if=1&search_wa_if=1&search_status=1&search_person1=&page='+str(page)

    driver.get(url)
    max_page = driver.find_element(By.XPATH, '/html/body/div[4]/div/div[2]/table[1]/tbody/tr/td[2]/div').text.split('/')[-1]
    print('max_page : ',max_page)
    return int(max_page)

In [3]:
options = webdriver.ChromeOptions() 
options.add_argument("headless")   #----show brower
driver = webdriver.Chrome('chromedriver',options=options) 

province = 'นนทบุรี'  #-----------------------edit here
d1 = datetime.today().strftime("%d%m%Y")
path = 'data'
file_name = f'{province}_{d1}.csv'
header = ['case_id','page','lot','sell_order','type_prop','size1','size2','size3','price','tumbon','aumper','province','Law_Court_ID','deed_no','addrno','bit_time1','bit_time2','bit_time3','bit_time4','bit_time5','bit_time6','link','timestamp']


In [ ]:
if not os.path.exists(path):
    os.makedirs(path) 

if not os.path.exists(f'{path}/{file_name}'):
    with open(f'{path}/{file_name}', 'w', encoding='UTF8') as f:
        writer = csv.writer(f,delimiter ='|')
        writer.writerow(header)

max_page = find_maxpage(province)

for page in range(1,max_page+1):
    read_table_data(province,page)
    

<ipython-input-1-c629e8e92aa1>:116: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('/html/body/div[4]/div/div/div/div/div[3]/div/div[1]/table/tbody/tr/td[2]/select').send_keys(province)


max_page :  47
page :  1


<ipython-input-1-c629e8e92aa1>:33: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  max_row = len(driver.find_element_by_xpath('/html/body/div[4]/div/div[2]/div[2]/table').text.split('\n'))-12
<ipython-input-1-c629e8e92aa1>:36: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  lot = driver.find_element_by_xpath(f'/html/body/div[4]/div/div[2]/div[2]/table/tbody/tr[{row}]/td[1]').text
<ipython-input-1-c629e8e92aa1>:37: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  sell_order = driver.find_element_by_xpath(f'/html/body/div[4]/div/div[2]/div[2]/table/tbody/tr[{row}]/td[2]').text
<ipython-input-1-c629e8e92aa1>:38: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  case_id = driver.find_element_by_xpath(f'/html/body/div[4]/div/div[2]/div[2]/table/tbody/tr[{row}]/td[3]').text
<ipython-input-1-c62

ผบ.96/2564 1/1 exist data--->pass
ผบ.253/2559 1/2 exist data--->pass
ผบ.1011/2562 1/3 exist data--->pass
ผบ.1193/2562 1/4 exist data--->pass
ผบ.2843/2562 1/5 exist data--->pass
ผบ.127/2564 1/6 exist data--->pass
ผบ.17106/2562 1/7 exist data--->pass
ผบE.600/2563 1/8 exist data--->pass
ผบ.1676/2563 1/9 exist data--->pass
พ.3050/2562 1/10 exist data--->pass
พ.3050/2562 1/11 exist data--->pass
พ.3050/2562 1/12 exist data--->pass
ผบ.23685/2562 1/13 exist data--->pass
ผบ.4525/2563 1/14 exist data--->pass
ผบ.5060/2562 1/15 exist data--->pass
ผบ.3998/2559 1/16 exist data--->pass
ผบE.232/2563 1/17 exist data--->pass
ผบ.3007/2558 1/18 exist data--->pass
ผบ.1477/2560 1/19 exist data--->pass
ผบ.1477/2560 1/20 exist data--->pass
ผบ.1338/2560 1/21 exist data--->pass
ผบ.1338/2560 1/22 exist data--->pass
ย.137/2558 1/23 exist data--->pass


# ค้นหา GPS
## find gps from exist data

In [5]:
import numpy as np
import pandas as pd

def find_gps_from_exist_file(path,deed_no):
    for file in os.listdir(path):
        if '.csv' in file:
            df2 = pd.read_csv(f'{path}/{file}', delimiter ='|')
            for index, row in df2.iterrows():
                if row['deed_no']==deed_no and str(row['lat'])!='nan':
                    if 'eva_unit' in row.keys():
                        return row['lat'],row['long'],row['eva_unit']
                    elif 'eval_unit_price' in row.keys():
                        return row['lat'],row['long'],row['eval_unit_price']
    return np.nan,np.nan,np.nan  

#make gps column if not exist
df = pd.read_csv(f'{path}/{file_name}', delimiter ='|')
if not 'lat' in df.keys():
    for i in ['lat','long','eva_unit']:
        if not i in df.keys():
            df[i] = np.nan
df.to_csv(f'{path}/{file_name}',sep='|',index=False)

#find gps from exist data
df = pd.read_csv(f'{path}/{file_name}', delimiter ='|')
for index, row in df.iterrows():
    if str(row['lat'])=='nan':
        lat,long,eva = find_gps_from_exist_file(path,row['deed_no'])
        print('update row:',index,row['case_id'],lat,long,eva)
        df['lat'][index] = lat
        df['long'][index] = long
        df['eva_unit'][index] = eva
    else:
        print('exist gps',row['case_id'])
df.to_csv(f'{path}/{file_name}',sep='|',index=False)
df

exist gps ผบ.96/2564
exist gps ผบ.253/2559
exist gps ผบ.1011/2562
exist gps ผบ.1193/2562
exist gps ผบ.2843/2562
exist gps ผบ.127/2564
exist gps ผบ.17106/2562
exist gps ผบE.600/2563
exist gps ผบ.1676/2563
exist gps พ.3050/2562
exist gps พ.3050/2562
exist gps พ.3050/2562
exist gps ผบ.23685/2562
exist gps ผบ.4525/2563
exist gps ผบ.5060/2562
exist gps ผบ.3998/2559
exist gps ผบE.232/2563
exist gps ผบ.3007/2558
exist gps ผบ.1477/2560
exist gps ผบ.1477/2560
exist gps ผบ.1338/2560
exist gps ผบ.1338/2560
exist gps ย.137/2558
exist gps ร.815/2562
exist gps ผบE.1820/2563
exist gps พ.2364/2561
exist gps มย.54/2563
exist gps ผบ.254/2557
exist gps ผบ.2615/2562
exist gps ผบ.1203/2563
exist gps ผบ.18191/2562
exist gps 5946/2560
exist gps ผบ.4961/2563
exist gps ผบ.5828/2559
exist gps ผบ.4409/2562
exist gps ผบ.4244/2561
exist gps ผบ.1296/2563
exist gps ผบ.2943/2563
exist gps ผบ.1076/2552
exist gps ผบ.910/2563
exist gps ผบ.4963/2559
exist gps ผบ.1740/2562
exist gps ผบ.4198/2563
exist gps ผบ.1953/2563
exi

<ipython-input-5-7a3a05b59133>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['lat'][index] = lat
<ipython-input-5-7a3a05b59133>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['long'][index] = long
<ipython-input-5-7a3a05b59133>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['eva_unit'][index] = eva


update row: 207 ผบ.1358/2563 nan nan nan
exist gps ผบ.11966/2555
exist gps ผบ.9245/2562
exist gps ผบ.5842/2563
exist gps ผบ.1614/2562
exist gps ผบ.4704/2561
exist gps ผบ.15355/2562
exist gps ผบ.14448/2562
exist gps ผบ.4439/2558
exist gps ผบ.3429/2563
exist gps พ.1481/2563
exist gps ผบ.5085/2561
exist gps พ.619/2560
exist gps ผบ.1642/2562
exist gps ผบ.14994/2561
exist gps ผบ.10569/2562
exist gps ผบ.12945/2562
exist gps ผบ.4195/2563
exist gps ผบ.975/2562
exist gps ผบ.2941/2562
exist gps ผบ.879/2560
exist gps ผบ.2107/2556
exist gps พ.379/2562
exist gps ผบ.5605/2563
exist gps ผบ.6465/2562
exist gps ผบE.1242/2563
exist gps ผบ.644/2560
exist gps ผบ.5057/2560
exist gps ผบ.373/2559
exist gps ผบ.17211/2562
exist gps ผบ.3443/2558
exist gps ผบ.4113/2559
exist gps ผบ.4597/2563
exist gps ผบE.1241/2563
exist gps ผบ.6113/2563
exist gps ผบ.329/2562
exist gps ผบ.4543/2560
exist gps ผบ.1063/2560
exist gps ผบ.8373/2561
exist gps ผบ.8593/2561
exist gps ย.203/2556
exist gps ผบ.3683/2557
exist gps ผบ.1763/2

update row: 766 11967/2542 nan nan nan
exist gps ธ.00475/2544
exist gps 14807/2537
exist gps 3139/2543
exist gps ย.481/2549
exist gps 7911/2548
exist gps ผบ.3311/2562
exist gps ผบ.7036/2561
exist gps ผบ.7081/2563
exist gps ผบ.3577/2562
exist gps ผบ.3766/2560
exist gps ผบ.2115/2562
exist gps พ.1100/2563
exist gps ผบ.17186/2561
exist gps ผบ.2502/2562
exist gps ผบ.1112/2558
exist gps ผบ.1631/2563
exist gps ผบ.456/2561
exist gps ผบ.1407/2554
exist gps ผบ.2088/2562
exist gps 7539/2552
exist gps ผบ.5806/2563
exist gps ผบ.2971/2563
exist gps ผบ.7090/2563
exist gps ผบ.18094/2562
exist gps ผบE.214/2563
exist gps ผบE.353/2563
exist gps ผบ.10013/2562
exist gps ผบE.487/2563
exist gps ผบ.1103/2561
exist gps ผบ.482/2556
exist gps ผบ.697/2560
exist gps ผบ.1177/2561
exist gps ผบ.4250/2563
exist gps ผบ.6012/2563
exist gps ผบ.1048/2563
exist gps ผบ.1981/2560
exist gps ผบ.1667/2562
exist gps ผบ.2364/2563
exist gps ผบ.1665/2563
exist gps ผบ.1457/2553
exist gps ผบ.683/2554
exist gps 10683/2543
exist gps 10

update row: 1173 พ.49/2558 nan nan nan
exist gps ผบ.4364/2563
exist gps ผบ.1389/2559
exist gps ผบ.14981/2560
exist gps ผบ.9008/2562
exist gps ผบ.3639/2556
exist gps ผบ.2055/2560
exist gps ผบ.5089/2560
exist gps ผบ.1082/2562
exist gps ผบ.11740/2562
exist gps ผบ.6648/2562
exist gps ผบ.11743/2562
exist gps ผบ.3760/2562
exist gps ผบ.1280/2563
exist gps ผบ.1280/2563
exist gps ผบ.2632/2563
exist gps ผบ.8137/2562
exist gps ผบ.14067/2559
exist gps พ.303/2563
exist gps ผบ.2114/2563
exist gps ผบ.5462/2562
exist gps มย.9/2563
exist gps ผบ.1079/2563
exist gps ผบ.4375/2562
exist gps ผบ.137/2556
exist gps ผบ.104/2555
exist gps ผบE.476/2563
exist gps ผบ.4357/2562
exist gps ผบ.4738/2563
exist gps ผบ.1685/2562
exist gps ผบ.1685/2562
exist gps ผบ.1621/2563
exist gps ผบ.4756/2563
exist gps ผบ.176/2560
exist gps ผบ.2566/2561
exist gps ผบ.2219/2563
exist gps ผบ.4187/2563
exist gps ผบ.1956/2563
exist gps ผบ.1867/2560
exist gps ผบ.15528/2561
exist gps ผบ.1307/2562
exist gps ผบ.567/2560
exist gps ผบ.15981/256

update row: 1942 ผบ.403/2555 nan nan nan
exist gps ย.324/2547
exist gps 5024/2550
exist gps 1297/2545
exist gps 2136/2536
exist gps 3696/2547
exist gps 142/2550
exist gps 07663/2541
exist gps 07663/2541
exist gps ย.1096/2549
exist gps ธ.3376/2546
exist gps 1424/2548
exist gps 13252/2544
exist gps 7774/2547
exist gps 11728/2551
exist gps 9804/2542
exist gps ธ.7526/2541
exist gps 624/2531
exist gps 624/2531
exist gps 624/2531
exist gps 3522/2541
exist gps 3522/2541
exist gps ผบ.787/2552
exist gps 12634/2541
exist gps 285/2549
exist gps 285/2549
exist gps 285/2549
exist gps ย.1959/2550
exist gps 19299/2540
exist gps ย.965/2547
exist gps 2031/2545
exist gps 1601/2541
exist gps ย.807/2551
exist gps 305/2543
exist gps 305/2543
exist gps 9259/2547
exist gps ย.5617/2541
exist gps ช.1185/2544
exist gps ช.1185/2544
exist gps 2219/2536
exist gps 4643/2542
exist gps ย.579/2547
exist gps 3921/2541
exist gps ย.385/2549
exist gps 2245/2544
exist gps ธ.10721/2543
exist gps ย.1703/2548
exist gps ย.1703

,case_id,page,lot,sell_order,type_prop,size1,size2,size3,price,tumbon,...,bit_time2,bit_time3,bit_time4,bit_time5,bit_time6,link,timestamp,lat,long,eva_unit
0,ผบ.96/2564,1/1,นนทบุรี 10/2565 - 1,8 - 1,ห้องชุด,0,0,30.59,1630447.0,บางเขน,...,2 28/04/2565 -\r\nคลิกเข้าร่วม,3 19/05/2565 -\r\nคลิกเข้าร่วม,4 09/06/2565 -\r\nคลิกเข้าร่วม,5 30/06/2565 -\r\nคลิกเข้าร่วม,6 21/07/2565 -\r\nคลิกเข้าร่วม,http://asset.led.go.th/newbidreg/asset_open.as...,2022-02-20 04:00:22.270263,13.844472,100.528327,NaN
1,ผบ.253/2559,1/2,นนทบุรี 10/2565 - 1,19 - 1,ที่ดินพร้อมสิ่งปลูกสร้าง,0,0,20.00,621295.0,บางตลาด,...,2 28/04/2565 -\r\nคลิกเข้าร่วม,3 19/05/2565 -\r\nคลิกเข้าร่วม,4 09/06/2565 -\r\nคลิกเข้าร่วม,5 30/06/2565 -\r\nคลิกเข้าร่วม,6 21/07/2565 -\r\nคลิกเข้าร่วม,http://asset.led.go.th/newbidreg/asset_open.as...,2022-02-20 04:00:24.229573,13.887363,100.522182,NaN
2,ผบ.1011/2562,1/3,นนทบุรี 10/2565 - 1,21 - 1,ที่ดินพร้อมสิ่งปลูกสร้าง,0,0,35.60,2139800.0,บางแม่นาง,...,2 28/04/2565 -\r\nคลิกเข้าร่วม,3 19/05/2565 -\r\nคลิกเข้าร่วม,4 09/06/2565 -\r\nคลิกเข้าร่วม,5 30/06/2565 -\r\nคลิกเข้าร่วม,6 21/07/2565 -\r\nคลิกเข้าร่วม,http://asset.led.go.th/newbidreg/asset_open.as...,2022-02-20 04:00:25.712786,13.857736,100.397070,NaN
3,ผบ.1193/2562,1/4,นนทบุรี 10/2565 - 1,22 - 1,ห้องชุด,0,0,26.49,1880790.0,เสาธงหิน,...,2 28/04/2565 -\r\nคลิกเข้าร่วม,3 19/05/2565 -\r\nคลิกเข้าร่วม,4 09/06/2565 -\r\nคลิกเข้าร่วม,5 30/06/2565 -\r\nคลิกเข้าร่วม,6 21/07/2565 -\r\nคลิกเข้าร่วม,http://asset.led.go.th/newbidreg/asset_open.as...,2022-02-20 04:00:27.402697,13.877734,100.413874,NaN
4,ผบ.2843/2562,1/5,นนทบุรี 10/2565 - 1,23 - 1,ห้องชุด,0,0,31.79,365585.0,บางใหญ่,...,2 28/04/2565 -\r\nคลิกเข้าร่วม,3 19/05/2565 -\r\nคลิกเข้าร่วม,4 09/06/2565 -\r\nคลิกเข้าร่วม,5 30/06/2565 -\r\nคลิกเข้าร่วม,6 21/07/2565 -\r\nคลิกเข้าร่วม,http://asset.led.go.th/newbidreg/asset_open.as...,2022-02-20 04:00:29.293760,13.843831,100.389356,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2300,ผบ.309/2563,47/1,วันเสาร์ 1/2565 - 1,206 - 1,ที่ดินพร้อมสิ่งปลูกสร้าง,0,0,24.90,1298400.0,บ้านใหม่,...,2 18/12/2564 งดขายส่งประกาศมิชอบ 18/12/2564,3 08/01/2565 งดขายส่งประกาศมิชอบ 08/01/2565,4 29/01/2565 งดขายส่งประกาศมิชอบ 29/01/2565,5 19/02/2565 -,6 12/03/2565 -\r\nคลิกเข้าร่วม,http://asset.led.go.th/newbidreg/asset_open.as...,2022-02-20 05:15:02.928349,13.950692,100.550542,nodetail
2301,ผบ.5147/2562,47/2,วันเสาร์ 1/2565 - 1,207 - 1,ห้องชุด,0,0,32.15,1852200.0,คลองเกลือ,...,2 18/12/2564 งดขายส่งประกาศมิชอบ 18/12/2564,3 08/01/2565 งดขายไม่มีผู้สู้ราคา 08/01/2565,4 29/01/2565 งดขายไม่มีผู้สู้ราคา 29/01/2565,5 19/02/2565 -,6 12/03/2565 -\r\nคลิกเข้าร่วม,http://asset.led.go.th/newbidreg/asset_open.as...,2022-02-20 05:15:04.767956,13.897168,100.552271,NaN
2302,ผบ.1989/2557,47/3,วันเสาร์ 1/2565 - 1,208 - 1,ที่ดินพร้อมสิ่งปลูกสร้าง,0,0,18.90,1592065.0,บางกร่าง,...,2 18/12/2564 งดขายส่งประกาศมิชอบ 18/12/2564,3 08/01/2565 งดขายไม่มีผู้สู้ราคา 08/01/2565,4 29/01/2565 ขายได้ 29/01/2565,5 19/02/2565 ขายได้ 29/01/2565,6 12/03/2565 ขายได้ 29/01/2565,http://asset.led.go.th/newbidreg/asset_open.as...,2022-02-20 05:15:07.107905,13.833896,100.434599,25000
2303,ผบE.119/2563,47/4,วันเสาร์ 1/2565 - 1,209 - 1,ที่ดินพร้อมสิ่งปลูกสร้าง,0,0,53.20,1587750.0,ลำโพ,...,2 18/12/2564 งดขาย 18/12/2564,3 08/01/2565 งดขาย 08/01/2565,4 29/01/2565 งดขาย 29/01/2565,5 19/02/2565 -,6 12/03/2565 -\r\nคลิกเข้าร่วม,http://asset.led.go.th/newbidreg/asset_open.as...,2022-02-20 05:15:08.844478,13.976577,100.405924,16000


# find gps from website

In [6]:
from urllib.parse import quote
from bs4 import BeautifulSoup
import requests
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import csv
import sys
import pandas as pd
from selenium.webdriver.support.ui import Select
import time 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import keyboard

start_row = 0   #-------------------------------edit
data_path = f'{path}/{file_name}'     #-------------------------------edit

def match_aumper(aumper):
    aumper_list = ['01-เมืองนนทบุรี','02-บางกรวย','03-บางใหญ่','04-บางบัวทอง','05-ไทรน้อย','06-ปากเกร็ด','บ1-บางปลา','ป1-ตลาดขวัญ','พ1-ตลิ่งชัน','พ7-ตลาดขวัญ','พ8-บางซื่อ']
    aumper = aumper.replace(' ','').split("(")[0]
    for a in aumper_list:
        if aumper in a:
            aumper = a
    return aumper

def find_gps(province,aumper,deed_no):
    #choose province
    element= Select(driver.find_element_by_xpath('/html/body/nav/form[1]/div/select'))
    element.select_by_visible_text(province)   # ddelement.select_by_value('12')

    #choose aumper
    aumper = match_aumper(aumper)
    time.sleep(1)
    element= Select(driver.find_element_by_xpath('/html/body/nav/form[2]/div/select'))
    element.select_by_visible_text(aumper)

    #insert deed number
    driver.find_element_by_xpath('/html/body/nav/form[3]/span/input').clear()
    driver.find_element_by_xpath('/html/body/nav/form[3]/span/input').send_keys(deed_no)
    
    #click button
    driver.find_element_by_xpath('/html/body/nav/form[4]/button').click()

    #gps window
    time.sleep(4)
    
    try:
        WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[3]/span/div/div[2]/div[2]/div/div[2]')))
    except:
        return None,None,None
    
    box = driver.find_element_by_xpath('/html/body/div[1]/div[3]/span/div/div[2]/div[2]/div/div[2]').text
    
    #sometime box show but not find deeed no ---->wait 4s and find deed no again 
    try:
        deed_no_from_box = box.split('\n')[1]
    except:
        time.sleep(4)
        box = driver.find_element_by_xpath('/html/body/div[1]/div[3]/span/div/div[2]/div[2]/div/div[2]').text
        deed_no_from_box = box.split('\n')[1]
        pass
        
    print('deed_no_from_box',deed_no_from_box,type(deed_no_from_box),'deed_no',deed_no,type(deed_no))

    if deed_no_from_box == deed_no:
        print('match.......deed_no_from_box == deed_no')
        gps = driver.find_element_by_xpath('/html/body/div[1]/div[3]/span/div/div[2]/div[2]/div/div[2]/div[10]/div[2]/a').text

        lat = gps.split(',')[0]
        long = gps.split(',')[1]
        eva_unit = driver.find_element_by_xpath('/html/body/div[1]/div[3]/span/div/div[2]/div[2]/div/div[2]/div[9]/div[2]')

        try:
            eva_unit = int(eva_unit.text.split()[0].replace(',',''))
        except:
            eva_unit = 'nodetail'
            pass
        print('lat',lat) 
        print('long',long)
        print('eva_unit',eva_unit)
        
        #print('close box window')
        driver.find_element_by_xpath('/html/body/div[1]/div[3]/span/div/div[2]/div[2]/div/div[12]/button[2]').click()
        
        return lat,long,eva_unit

#init-----------------------------------------------------
options = webdriver.ChromeOptions() 
# options.add_argument("headless")
driver = webdriver.Chrome('chromedriver',options=options)

url = 'https://landsmaps.dol.go.th/'
driver.get(url)

try:
    #close popup
    print('close popup')
    time.sleep(3)
    WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[25]/div/div/div/div[1]/button/i')))
    driver.find_element_by_xpath('/html/body/div[25]/div/div/div/div[1]/button/i').click()
    
except:
    print('no close popup-->pass')
    pass

#test--------------------------------------------------------------------------------
province = 'นนทบุรี'
aumper = 'บางใหญ่'
deed_no = '58211'
gps = find_gps(province,aumper,deed_no)
if gps:
    print('Can find GPS...Work!')
    
#loop find gps-------------------------------------------------------------
df = pd.read_csv(data_path, delimiter ='|')

for index, row in df.iterrows():
    
#     if keyboard.is_pressed("q"):
#         print("q pressed, ending loop")
#         break
    
    try:
        print(index)
    #     if index == 100:
    #         break

        province = row['province']
        aumper = row['aumper']
        deed_nos = row['deed_no'].split(',')

        if pd.isna(row['lat']) and index >= start_row:
            print('row["lat"] is Null........................................')
            
            for deed_no in deed_nos:
                print('row["lat"],province,aumper,deed_no',row['lat'],province,aumper,deed_no)
                lat,long,eva_unit = find_gps(province,aumper,deed_no)
                
                if lat:
                    df['lat'][index] = lat
                    df['long'][index] = long
                    df['eva_unit'][index] = eva_unit

                    df.to_csv(data_path,sep='|',index=False)
                    break
            else:
                print('not found GPS Data...........',row['link'])

        else:
            print('have row["lat"] pass.......................................')
            
    except:
        print('error!!!!!!!!!!!',row['link'])
        pass
    


KeyboardInterrupt: 

# Test

In [7]:
# import pandas as pd
df = pd.read_csv(f'{path}/{file_name}', delimiter ='|')
df

,case_id,page,lot,sell_order,type_prop,size1,size2,size3,price,tumbon,...,bit_time2,bit_time3,bit_time4,bit_time5,bit_time6,link,timestamp,lat,long,eva_unit
0,ผบ.96/2564,1/1,นนทบุรี 10/2565 - 1,8 - 1,ห้องชุด,0,0,30.59,1630447.0,บางเขน,...,2 28/04/2565 -\r\nคลิกเข้าร่วม,3 19/05/2565 -\r\nคลิกเข้าร่วม,4 09/06/2565 -\r\nคลิกเข้าร่วม,5 30/06/2565 -\r\nคลิกเข้าร่วม,6 21/07/2565 -\r\nคลิกเข้าร่วม,http://asset.led.go.th/newbidreg/asset_open.as...,2022-02-20 04:00:22.270263,13.844472,100.528327,NaN
1,ผบ.253/2559,1/2,นนทบุรี 10/2565 - 1,19 - 1,ที่ดินพร้อมสิ่งปลูกสร้าง,0,0,20.00,621295.0,บางตลาด,...,2 28/04/2565 -\r\nคลิกเข้าร่วม,3 19/05/2565 -\r\nคลิกเข้าร่วม,4 09/06/2565 -\r\nคลิกเข้าร่วม,5 30/06/2565 -\r\nคลิกเข้าร่วม,6 21/07/2565 -\r\nคลิกเข้าร่วม,http://asset.led.go.th/newbidreg/asset_open.as...,2022-02-20 04:00:24.229573,13.887363,100.522182,NaN
2,ผบ.1011/2562,1/3,นนทบุรี 10/2565 - 1,21 - 1,ที่ดินพร้อมสิ่งปลูกสร้าง,0,0,35.60,2139800.0,บางแม่นาง,...,2 28/04/2565 -\r\nคลิกเข้าร่วม,3 19/05/2565 -\r\nคลิกเข้าร่วม,4 09/06/2565 -\r\nคลิกเข้าร่วม,5 30/06/2565 -\r\nคลิกเข้าร่วม,6 21/07/2565 -\r\nคลิกเข้าร่วม,http://asset.led.go.th/newbidreg/asset_open.as...,2022-02-20 04:00:25.712786,13.857736,100.397070,NaN
3,ผบ.1193/2562,1/4,นนทบุรี 10/2565 - 1,22 - 1,ห้องชุด,0,0,26.49,1880790.0,เสาธงหิน,...,2 28/04/2565 -\r\nคลิกเข้าร่วม,3 19/05/2565 -\r\nคลิกเข้าร่วม,4 09/06/2565 -\r\nคลิกเข้าร่วม,5 30/06/2565 -\r\nคลิกเข้าร่วม,6 21/07/2565 -\r\nคลิกเข้าร่วม,http://asset.led.go.th/newbidreg/asset_open.as...,2022-02-20 04:00:27.402697,13.877734,100.413874,NaN
4,ผบ.2843/2562,1/5,นนทบุรี 10/2565 - 1,23 - 1,ห้องชุด,0,0,31.79,365585.0,บางใหญ่,...,2 28/04/2565 -\r\nคลิกเข้าร่วม,3 19/05/2565 -\r\nคลิกเข้าร่วม,4 09/06/2565 -\r\nคลิกเข้าร่วม,5 30/06/2565 -\r\nคลิกเข้าร่วม,6 21/07/2565 -\r\nคลิกเข้าร่วม,http://asset.led.go.th/newbidreg/asset_open.as...,2022-02-20 04:00:29.293760,13.843831,100.389356,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2300,ผบ.309/2563,47/1,วันเสาร์ 1/2565 - 1,206 - 1,ที่ดินพร้อมสิ่งปลูกสร้าง,0,0,24.90,1298400.0,บ้านใหม่,...,2 18/12/2564 งดขายส่งประกาศมิชอบ 18/12/2564,3 08/01/2565 งดขายส่งประกาศมิชอบ 08/01/2565,4 29/01/2565 งดขายส่งประกาศมิชอบ 29/01/2565,5 19/02/2565 -,6 12/03/2565 -\r\nคลิกเข้าร่วม,http://asset.led.go.th/newbidreg/asset_open.as...,2022-02-20 05:15:02.928349,13.950692,100.550542,nodetail
2301,ผบ.5147/2562,47/2,วันเสาร์ 1/2565 - 1,207 - 1,ห้องชุด,0,0,32.15,1852200.0,คลองเกลือ,...,2 18/12/2564 งดขายส่งประกาศมิชอบ 18/12/2564,3 08/01/2565 งดขายไม่มีผู้สู้ราคา 08/01/2565,4 29/01/2565 งดขายไม่มีผู้สู้ราคา 29/01/2565,5 19/02/2565 -,6 12/03/2565 -\r\nคลิกเข้าร่วม,http://asset.led.go.th/newbidreg/asset_open.as...,2022-02-20 05:15:04.767956,13.897168,100.552271,NaN
2302,ผบ.1989/2557,47/3,วันเสาร์ 1/2565 - 1,208 - 1,ที่ดินพร้อมสิ่งปลูกสร้าง,0,0,18.90,1592065.0,บางกร่าง,...,2 18/12/2564 งดขายส่งประกาศมิชอบ 18/12/2564,3 08/01/2565 งดขายไม่มีผู้สู้ราคา 08/01/2565,4 29/01/2565 ขายได้ 29/01/2565,5 19/02/2565 ขายได้ 29/01/2565,6 12/03/2565 ขายได้ 29/01/2565,http://asset.led.go.th/newbidreg/asset_open.as...,2022-02-20 05:15:07.107905,13.833896,100.434599,25000
2303,ผบE.119/2563,47/4,วันเสาร์ 1/2565 - 1,209 - 1,ที่ดินพร้อมสิ่งปลูกสร้าง,0,0,53.20,1587750.0,ลำโพ,...,2 18/12/2564 งดขาย 18/12/2564,3 08/01/2565 งดขาย 08/01/2565,4 29/01/2565 งดขาย 29/01/2565,5 19/02/2565 -,6 12/03/2565 -\r\nคลิกเข้าร่วม,http://asset.led.go.th/newbidreg/asset_open.as...,2022-02-20 05:15:08.844478,13.976577,100.405924,16000


In [8]:
df['lat'].isna().sum()

12

# Map
## Circle Layer

In [ ]:
from folium import plugins
import folium
import matplotlib.pyplot as plt
import random
import pandas as pd

def int_to_number(a):
    a = str(a).split('.')[0]
    x = [a[-9:-6],a[-6:-3],a[-3:]]
    z = ''
    for index,i in enumerate(x):
        if i != '':
            z += i 
            if index != 2:
                z+=','
    return z 

def make_circle_map(df):
    latitude = df['lat'].mean()
    longitude = df['long'].mean()

    deed_map = folium.Map(location = [latitude, longitude], zoom_start = 12)

    #creat group
    colors = [
        'red',
        'blue',
        'gray',
        'darkred',
        'lightred',
        'orange',
        'beige',
        'green',
        'darkgreen',
        'lightgreen',
        'darkblue',
        'lightblue',
        'purple',
        'darkpurple',
        'pink',
        'cadetblue',
        'lightgray',
        'black'
    ]
    list_group = df['type_prop'].unique().tolist()
#     list_colors = [ random.choice(colors) for i in range(len(list_group))]
    list_colors = ['blue','red','darkgreen','gray']
    groups = {}
    for group in list_group:
        groups[group] = folium.FeatureGroup(name=group).add_to(deed_map)

    
    folium.TileLayer(
      tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
      attr = 'Esri',
      name = 'Satellite',
      ).add_to(deed_map)
    
    folium.TileLayer('openstreetmap').add_to(deed_map)
    folium.LayerControl().add_to(deed_map)

    for index, row in df.iterrows():
        def find_last_sta(bit_texts):
            def find_avi(bit_texts):
                for i,bit_text in enumerate(bit_texts):
                    for s in ['คลิกเข้าร่วม','ขายได้']:
                        if s in bit_text:
                            return s,bit_text

                return None,None

            def find_sta(bit_texts):
                for i,bit_text in enumerate(bit_texts):
                    stas = ['ถอนการยึด','งดขายส่งประกาศมิชอบ','งดขายไม่มีผู้สู้ราคา','โจทก์แถลงงดขาย','เจ้าพนักงานงดขาย','งดขาย','ขายได้']
                    for s in stas:
                        if s in bit_text:
                            return s,bit_text

                return None,None

            a,b = find_avi(bit_texts)
            if not a:
                bit_texts.reverse()
                a,b = find_sta(bit_texts)  
            return a,b.split()[0],b.split()[1]
        
        bit_texts = [row['bit_time1'],row['bit_time2'],row['bit_time3'],row['bit_time4'],row['bit_time5'],row['bit_time6']]
        staus,staus_no,staus_date = find_last_sta(bit_texts)

        lng = row['long']
        lat = row['lat']
        group = row['type_prop']
        label = row['label']
        price = int_to_number(row['price'])
        insta_post = row['label']+'#lg=1&slide=0'
        website = row['label']
        direction = f"https://www.google.co.th/maps/search/{row['lat']},+{row['long']}"
        size = [row['size1'],row['size2'],row['size3']]

        pub_html = folium.Html(f"""<p style="text-align: center;"><span style="font-family: Didot, serif; font-size: 21px;"></span></p>
        <p style="text-align: center;"><iframe src={insta_post}embed width="380" height="220" frameborder="0" scrolling="false" allowtransparency="true"></iframe>
        <p style="text-align: center;"><a href={website} target="_blank" title="website"><span style="font-family: Didot, serif; font-size: 17px;">{group} {size[0]}ไร่ {size[1]}งาน {size[2]}ตร.วา</span></a></p>
         <p style="text-align: center;"><a href={direction} target="_blank" title="นำทาง"><span style="font-family: Didot, serif; font-size: 17px;">Status: {staus} นัด {staus_no} {staus_date}</span></a></p>
        <p style="text-align: center;"><a href={direction} target="_blank" title="นำทาง"><span style="font-family: Didot, serif; font-size: 17px;">{price} บาท</span></a></p>
        """, script=True)
        popup = folium.Popup(pub_html, max_width=700)

        if str(lat)!='nan':
            n = list_group.index(group)
            color = list_colors[n]
        
            if staus == "คลิกเข้าร่วม":
                color_border = 'yellow'
                fill_opacity = 0.8
            else:
                color_border = 'grey'
                fill_opacity = 0.2
                
            if 'onweb' in list(df.keys()):
                if row['onweb'] == False:
                    color_border = 'grey'
                    fill_opacity = 0.2
                
            if group=='ที่ดินพร้อมสิ่งปลูกสร้าง' and int(row['size1'])==0 and int(row['size2'])==0 and int(row['size3'])<40:
                color = 'purple'

            groups[group].add_child(
                folium.CircleMarker(
                     [lat, lng],
                     radius=7, 
                     color=color_border,
                     fill=True,
                     fill_color=color,
                     fill_opacity=fill_opacity,
                     popup=popup
                 )
            )
        
    return deed_map

df = pd.read_csv(f'{path}/{file_name}', delimiter ='|')
df['label'] = df['link']
deed_map = make_circle_map(df)  #lat,long,type_prop,label
deed_map.save(f"{path}/{file_name.split('.')[0]}_circle_map.html")
deed_map

## Incidents

In [ ]:
#okkkk
from folium import plugins
import folium
import matplotlib.pyplot as plt
import random
import pandas as pd

def make_incidents_map(df):
    latitude = df['lat'].mean()
    longitude = df['long'].mean()
    
    sanfran_map = folium.Map(location = [latitude, longitude], zoom_start = 12)
    incidents = plugins.MarkerCluster().add_to(sanfran_map)
    
    list_group = df['type_prop'].unique().tolist()
#     list_colors = [ random.choice(colors) for i in range(len(list_group))]
    list_colors = ['darkblue','red','green','black']
    
    
    folium.TileLayer('openstreetmap').add_to(sanfran_map)
#     folium.TileLayer('Stamen Terrain').add_to(sanfran_map)
#     folium.LayerControl(collapsed=False).add_to(sanfran_map)
#     folium.LayerControl().add_to(sanfran_map)
    
    def int_to_number(a):
        a = str(a).split('.')[0]
        x = [a[-9:-6],a[-6:-3],a[-3:]]
        z = ''
        for index,i in enumerate(x):
            if i != '':
                z += i 
                if index != 2:
                    z+=','
        return z    

    for index, row in df.iterrows():
        lng = row['long']
        lat = row['lat']
        group = row['type_prop']
        label = row['label']
        price = int_to_number(row['price'])
        
        
        insta_post = row['label']+'#lg=1&slide=0'
        website = row['label']
#         direction = f"https://www.google.co.th/maps/@{row['lat']},{row['long']},15z"
#         https://www.google.co.th/maps/search/13.662452,+100.293810
        direction = f"https://www.google.co.th/maps/search/{row['lat']},+{row['long']}"
        size = [row['size1'],row['size2'],row['size3']]
    
        
#         pub_html = folium.Html(f"""<p style="text-align: center;"><span style="font-family: Didot, serif; font-size: 21px;">{name}</span></p>
#         <p style="text-align: center;"><iframe src={insta_post}embed width="240" height="290" frameborder="0" scrolling="auto" allowtransparency="true"></iframe>
#         <p style="text-align: center;"><a href={website} target="_blank" title="{name} Website"><span style="font-family: Didot, serif; font-size: 17px;">{name} Website</span></a></p>
#         <p style="text-align: center;"><a href={directions} target="_blank" title="Directions to {name}"><span style="font-family: Didot, serif; font-size: 17px;">Directions to {name}</span></a></p>
#         """, script=True)

        pub_html = folium.Html(f"""<p style="text-align: center;"><span style="font-family: Didot, serif; font-size: 21px;"></span></p>
        <p style="text-align: center;"><iframe src={insta_post}embed width="380" height="220" frameborder="0" scrolling="false" allowtransparency="true"></iframe>
        <p style="text-align: center;"><a href={website} target="_blank" title="website"><span style="font-family: Didot, serif; font-size: 17px;">{group} {size[0]}ไร่ {size[1]}งาน {size[2]}ตร.วา</span></a></p>
        <p style="text-align: center;"><a href={direction} target="_blank" title="นำทาง"><span style="font-family: Didot, serif; font-size: 17px;">{price} บาท</span></a></p>
        """, script=True)
    
        # Create pop-up with html content
        popup = folium.Popup(pub_html, max_width=700)
    
        n = list_group.index(group)
        color = list_colors[n]
        
        if str(lat)!='nan':
            folium.Marker(
                location = [lat, lng],
                icon=folium.Icon(color=color),
                markerColor=color,
                popup=popup,
            ).add_to(incidents)
            
    return sanfran_map

df = pd.read_csv(f'{path}/{file_name}', delimiter ='|')
df['label'] = df['link']
deed_map = make_incidents_map(df)  #lat,long,type_prop,label
deed_map.save(f"{path}/{file_name.split('.')[0]}_incidents_map.html")
deed_map


# combine data and make map

In [ ]:
def find_order_file(path):
    d = []
    for file in os.listdir(path):
        if '.csv' in file:
            print(file)
            date = file.split('_')[-1].split('.')[0]
            print(date)
            my_date = datetime.strptime(date, "%d%m%Y")
            d.append(my_date)
    d.sort()
    d.reverse()

    order_file = []
    for dd in d:
        for file in os.listdir(path):
            if '.csv' in file:
                date = file.split('_')[-1].split('.')[0]
                date = datetime.strptime(date, "%d%m%Y")
                if date==dd:
                    order_file.append(file)
    return order_file

files = find_order_file(path)



combile_df = pd.read_csv(f'{path}/{files[0]}', delimiter ='|')
print(combile_df.shape,combile_df.keys())

for f in files[1:]:
    df2 = pd.read_csv(f'{path}/{f}', delimiter ='|')
    for index2, row2 in df2.iterrows():
        exist = False
        for index, row in combile_df.iterrows():
            if row2['case_id']==row['case_id']:
                exist = True
                print('exist',row2['case_id'])
        
        if not exist:
            print('not exist append',row2['case_id'])
            combile_df = combile_df.append(row2, ignore_index = True)
            
current_df = pd.read_csv(f'{path}/{files[0]}', delimiter ='|')

onweb = []
for c in list(combile_df['case_id']):
    if c in list(current_df['case_id']):
        onweb.append(True)
    else:
        onweb.append(False)
combile_df['onweb'] = onweb

combile_df.to_csv(f'{path}/combile_df_{file_name}',sep='|',index=False)


In [ ]:
combile_df['label'] = combile_df['link']

deed_map = make_circle_map(combile_df)  #lat,long,type_prop,label
deed_map.save(f"{path}/combile_{file_name.split('.')[0]}_circle_map.html")
deed_map

In [ ]:
deed_map = make_incidents_map(combile_df)  #lat,long,type_prop,label
deed_map.save(f"{path}/combile_{file_name.split('.')[0]}_incidents_map.html")
deed_map